In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('n50.csv')

In [3]:
def number_of_years(y):  # calculates the number of years of the dataset
    p = y.index[0]  # date of first row in the dataset (datetime format)
    q = y.index[len(y) - 1]  # date of last row in the dataset  (datetime format)
    return ((q - p).days + 1) / 365  # the difference give the number of total days (not trading days) over the total number of years in the dataset

In [4]:
def ratio(a, b, c):  # function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
    return (a - c) / b  # a => annual return, c => risk_free_rate, b => deviation (standard for sharpe, semi for sortino)


In [5]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights


In [6]:
mvo_sortino=pd.DataFrame(columns=['Date', 'Returns', 'Risk','Ratio'])
inputweights=[]

In [7]:
def MVO_Sortino(start_date,end_date,mvo_sortino):
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[start_date:end_date]
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    #trading_days = len(df) / number_of_years(tdf)
    trading_days=60
    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio
    returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
    for cols in returnso.columns.tolist():
        for i in range(0, len(df)):
            if returnso[cols][i] > 0:
                returnso[cols][i] = 0  
    covmatso = returnso.cov() * trading_days
    num_portfolios = 50000                   #initializing number of portfolios to 50000; referred from Wang et al (2020) (science direct)
    num_assets = len(df.columns)              #initializing number of stocks/assets considered in the portfolio
    risk_free_rate = 0.0358 
    portfolio_returns = []                    #initializing an empty list for portfolio returns
    portfolio_volatility =[]                  #initializing an empty list for portfolio risk
    stock_weights =[]                         #initializing an empty list for portfolio weights
    sortino =[]          
    for single_portfolio in range(5):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)                                    #No Short Selling Allowed => weights add up to 1   "x = x+y" => "x+=y"    weights = weights/np.sum(weights)  
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
        varso=np.dot(weights.T,np.dot(covmatso,weights))
        semi_temp = np.sqrt(varso)
        sotemp = ratio(returns_temp,semi_temp,risk_free_rate)   #calculating sharpe ratio
        portfolio_returns.append(returns_temp)                       
        portfolio_volatility.append(semi_temp)
        stock_weights.append(weights)
        sortino.append(sotemp)
    portfolio = {'Returns' : portfolio_returns, 'Standard Deviation' : portfolio_volatility, 'Sortino Ratio' : sortino}
    for counter,symbol in enumerate(df.columns):
        portfolio[symbol + " Weight"] = [Weight[counter] for Weight in stock_weights]
    pc = pd.DataFrame(portfolio)
    pc.loc[:, :] *= 100
    pc['Sortino Ratio']=pc['Sortino Ratio']/100 
    max_sortino=pc['Sortino Ratio'].max()                                             #Best optimised portfolio wrt sharpe ratio
    max_sortino_portfolio=pc.loc[pc['Sortino Ratio'] == max_sortino]
    max_weights=max_sortino_portfolio.values
    max_weights=max_weights[0][3:]
    return max_weights

In [8]:
def MVO_Sortino_lookback(start_date,end_date,deciding_date,mvo_sortino,inputweights):
    mvo_sortino_list=[]
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[start_date:end_date]
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    #trading_days = len(df) / number_of_years(tdf)
    trading_days=60
    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio
    returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
    for cols in returnso.columns.tolist():
        for i in range(0, len(df)):
            if returnso[cols][i] > 0:
                returnso[cols][i] = 0  
    covmatsh = returnsh.cov() * trading_days  # Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
    covmatso = returnso.cov() * trading_days  # Annualised covariance matrix calculated wrt returnso i.e. used to calculate sortino ratio
    num_assets = len(df.columns)  # initializing number of stocks/assets considered in the portfolio
    risk_free_rate = 0.0358  # initializing risk free rate that will be used in calculating both the ratios (absolute value)
# referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
# In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

# 2021_chen etal_Mean–variance portfolio optimization using machine learning-based stock price prediction
# Repeat the process 50,000times. From a statistical point of view, 50,000 random portfolios cover most possible portfolios with different weights and aresufficiently representative

    global_war_sortino=[]
    sortino_portfolio_return=[]
    sortino_portfolio_risk=[]
    sortino_portfolio_soratio=[]
    sortino_portfolio_stockWeights=[]
    fitness=0
    weights=np.array(BALANCE(np.array(inputweights)))
    returns_temp = np.sum(returnso.mean()*weights)*trading_days 
    varso=np.dot(weights.T,np.dot(covmatso,weights))   
    semi_temp = np.sqrt(varso)      
    fitness = ratio(returns_temp,semi_temp,risk_free_rate)
    sortino_portfolio_return.append(returns_temp)
    sortino_portfolio_risk.append(semi_temp)
    sortino_portfolio_soratio.append(fitness)
    sortino_portfolio_stockWeights.append(weights)
    sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  
    for counter,symbol in enumerate(df.columns):
      sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
    sortino_pc = pd.DataFrame(sortino_portfolio)
    sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
    sortino_optimal=sortino_optimal.to_frame()
    sortino_optimal=sortino_optimal.transpose()
    sortino_optimal.loc[:, :] *= 100
    sortino_optimal.loc[:, 'Sortino Ratio'] /= 100
    to_append=[deciding_date,sortino_optimal['Returns'].values[0],sortino_optimal['Semi Deviation'].values[0],sortino_optimal['Sortino Ratio'].values[0]]
    mvo_sortino_series = pd.Series(to_append, index = mvo_sortino.columns)
    mvo_sortino = mvo_sortino.append(mvo_sortino_series, ignore_index=True)
    sortino_optimal.to_csv(str(deciding_date)+"sortino_optimal.csv")
    print(mvo_sortino)
    return mvo_sortino


In [9]:
start_date="2016-01-01"

In [10]:
start_date_index=df.index[df['Date'] == start_date]

In [11]:
end_date="2020-09-30"

In [12]:
end_date_index=df.index[df['Date']== end_date]

In [13]:
start_date_lookback="2020-10-01"

In [14]:
start_date_lookback_index=df.index[df['Date'] == start_date_lookback]

In [15]:
end_date_lookback="2020-12-31"

In [16]:
end_date_lookback_index=df.index[df['Date']== end_date_lookback]


In [ ]:
while df.iloc[end_date_index].Date.values[0]:
    inputweights=MVO_Sortino(df.iloc[start_date_index].Date.values[0],df.iloc[end_date_index].Date.values[0],mvo_sortino)
    mvo_sortino=MVO_Sortino_lookback(df.iloc[start_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index+1].Date.values[0],mvo_sortino,inputweights)
    start_date_lookback_index+=1
    end_date_lookback_index+=1
    start_date_index+=1
    end_date_index+=1

         Date    Returns      Risk     Ratio
0  2021-01-01 -32.719553  4.546724 -7.983672
         Date    Returns      Risk     Ratio
0  2021-01-01 -32.719553  4.546724 -7.983672
1  2021-01-04 -30.923559  4.114551 -8.385740
         Date    Returns      Risk     Ratio
0  2021-01-01 -32.719553  4.546724 -7.983672
1  2021-01-04 -30.923559  4.114551 -8.385740
2  2021-01-05 -31.107013  4.295521 -8.075159
         Date    Returns      Risk     Ratio
0  2021-01-01 -32.719553  4.546724 -7.983672
1  2021-01-04 -30.923559  4.114551 -8.385740
2  2021-01-05 -31.107013  4.295521 -8.075159
3  2021-01-06 -31.477541  4.619192 -7.589540
         Date    Returns      Risk     Ratio
0  2021-01-01 -32.719553  4.546724 -7.983672
1  2021-01-04 -30.923559  4.114551 -8.385740
2  2021-01-05 -31.107013  4.295521 -8.075159
3  2021-01-06 -31.477541  4.619192 -7.589540
4  2021-01-07 -31.346510  4.472265 -7.809580
         Date    Returns      Risk     Ratio
0  2021-01-01 -32.719553  4.546724 -7.983672
1  2021-01

          Date    Returns      Risk     Ratio
0   2021-01-01 -32.719553  4.546724 -7.983672
1   2021-01-04 -30.923559  4.114551 -8.385740
2   2021-01-05 -31.107013  4.295521 -8.075159
3   2021-01-06 -31.477541  4.619192 -7.589540
4   2021-01-07 -31.346510  4.472265 -7.809580
5   2021-01-08 -30.828805  4.232523 -8.129619
6   2021-01-11 -29.411776  4.260063 -7.744434
7   2021-01-12 -29.885259  3.866710 -8.654712
8   2021-01-13 -30.495145  4.197511 -8.117940
9   2021-01-14 -28.221531  3.774977 -8.424299
10  2021-01-15 -29.774235  4.114553 -8.106405
11  2021-01-18 -30.999820  3.992985 -8.660144
12  2021-01-19 -32.021927  4.369964 -8.146961
13  2021-01-20 -30.894847  4.303252 -8.011347
14  2021-01-21 -30.411978  4.099495 -8.291748
15  2021-01-22 -32.085974  4.490195 -7.943080
16  2021-01-25 -31.168495  4.383952 -7.926295
17  2021-01-27 -30.841193  4.236317 -8.125263
18  2021-01-28 -32.651460  4.642578 -7.804168
          Date    Returns      Risk     Ratio
0   2021-01-01 -32.719553  4.54672

          Date    Returns      Risk     Ratio
0   2021-01-01 -32.719553  4.546724 -7.983672
1   2021-01-04 -30.923559  4.114551 -8.385740
2   2021-01-05 -31.107013  4.295521 -8.075159
3   2021-01-06 -31.477541  4.619192 -7.589540
4   2021-01-07 -31.346510  4.472265 -7.809580
5   2021-01-08 -30.828805  4.232523 -8.129619
6   2021-01-11 -29.411776  4.260063 -7.744434
7   2021-01-12 -29.885259  3.866710 -8.654712
8   2021-01-13 -30.495145  4.197511 -8.117940
9   2021-01-14 -28.221531  3.774977 -8.424299
10  2021-01-15 -29.774235  4.114553 -8.106405
11  2021-01-18 -30.999820  3.992985 -8.660144
12  2021-01-19 -32.021927  4.369964 -8.146961
13  2021-01-20 -30.894847  4.303252 -8.011347
14  2021-01-21 -30.411978  4.099495 -8.291748
15  2021-01-22 -32.085974  4.490195 -7.943080
16  2021-01-25 -31.168495  4.383952 -7.926295
17  2021-01-27 -30.841193  4.236317 -8.125263
18  2021-01-28 -32.651460  4.642578 -7.804168
19  2021-01-29 -32.867573  4.570160 -7.975120
20  2021-02-01 -32.420420  4.40720

          Date    Returns      Risk     Ratio
0   2021-01-01 -32.719553  4.546724 -7.983672
1   2021-01-04 -30.923559  4.114551 -8.385740
2   2021-01-05 -31.107013  4.295521 -8.075159
3   2021-01-06 -31.477541  4.619192 -7.589540
4   2021-01-07 -31.346510  4.472265 -7.809580
5   2021-01-08 -30.828805  4.232523 -8.129619
6   2021-01-11 -29.411776  4.260063 -7.744434
7   2021-01-12 -29.885259  3.866710 -8.654712
8   2021-01-13 -30.495145  4.197511 -8.117940
9   2021-01-14 -28.221531  3.774977 -8.424299
10  2021-01-15 -29.774235  4.114553 -8.106405
11  2021-01-18 -30.999820  3.992985 -8.660144
12  2021-01-19 -32.021927  4.369964 -8.146961
13  2021-01-20 -30.894847  4.303252 -8.011347
14  2021-01-21 -30.411978  4.099495 -8.291748
15  2021-01-22 -32.085974  4.490195 -7.943080
16  2021-01-25 -31.168495  4.383952 -7.926295
17  2021-01-27 -30.841193  4.236317 -8.125263
18  2021-01-28 -32.651460  4.642578 -7.804168
19  2021-01-29 -32.867573  4.570160 -7.975120
20  2021-02-01 -32.420420  4.40720

          Date    Returns      Risk     Ratio
0   2021-01-01 -32.719553  4.546724 -7.983672
1   2021-01-04 -30.923559  4.114551 -8.385740
2   2021-01-05 -31.107013  4.295521 -8.075159
3   2021-01-06 -31.477541  4.619192 -7.589540
4   2021-01-07 -31.346510  4.472265 -7.809580
5   2021-01-08 -30.828805  4.232523 -8.129619
6   2021-01-11 -29.411776  4.260063 -7.744434
7   2021-01-12 -29.885259  3.866710 -8.654712
8   2021-01-13 -30.495145  4.197511 -8.117940
9   2021-01-14 -28.221531  3.774977 -8.424299
10  2021-01-15 -29.774235  4.114553 -8.106405
11  2021-01-18 -30.999820  3.992985 -8.660144
12  2021-01-19 -32.021927  4.369964 -8.146961
13  2021-01-20 -30.894847  4.303252 -8.011347
14  2021-01-21 -30.411978  4.099495 -8.291748
15  2021-01-22 -32.085974  4.490195 -7.943080
16  2021-01-25 -31.168495  4.383952 -7.926295
17  2021-01-27 -30.841193  4.236317 -8.125263
18  2021-01-28 -32.651460  4.642578 -7.804168
19  2021-01-29 -32.867573  4.570160 -7.975120
20  2021-02-01 -32.420420  4.40720